## Testing Geoapify Places API

In [1]:
import sys
import pandas as pd
import numpy as np
# change working directory to the root of the project
sys.path.append('../')
from src.geoapify_api import geoapify_pipeline
from src.methods.data_ingestion import convert_easting_northing_to_long_lat
import importlib

In [2]:
importlib.reload(sys.modules['src.geoapify_api'])
request_params = {
    "apiKey": "493bbfea1f124b27906cf052aa330680",
    "categories": "tourism",
    "limit": "500",
}
filters = {
        "circle": "-1.1549,52.9545,5000",
}

url = 'https://api.geoapify.com/v2/places?'

df, res = geoapify_pipeline(url, request_params, filters)
df

,name,country,country_code,state,city,postcode,district,suburb,street,lon,...,town,housenumber,commercial,sourcename,attribution,license,url,raw,website,category_keywords
0,Nottingham Castle Museum & Art Gallery,United Kingdom,gb,England,Nottingham,NG1 6EL,Lace Market,The Meadows,Castle Road,-1.154333,...,NaN,NaN,NaN,openstreetmap,© OpenStreetMap contributors,Open Database Licence,https://www.openstreetmap.org/copyright,{'name': 'Nottingham Castle Museum & Art Galle...,https://www.nottinghamcastle.org.uk/,"[historic, sights, tourism, museum, entertainm..."
1,Stonebridge City Farm,United Kingdom,gb,England,Nottingham,NG3 2FR,NaN,St Ann's,Ferrers Walk,-1.133927,...,NaN,NaN,NaN,openstreetmap,© OpenStreetMap contributors,Open Database Licence,https://www.openstreetmap.org/copyright,"{'name': 'Stonebridge City Farm', 'email': 'in...",https://stonebridgecityfarm.com/,"[tourism, attraction]"
2,Ye Olde Trip to Jerusalem,United Kingdom,gb,England,Nottingham,NG1 6AD,Lace Market,The Meadows,Castle Road,-1.152649,...,NaN,NaN,NaN,openstreetmap,© OpenStreetMap contributors,Open Database Licence,https://www.openstreetmap.org/copyright,"{'name': 'Ye Olde Trip to Jerusalem', 'image':...",http://www.triptojerusalem.com/,"[historic, sights, tourism, heritage, building..."
3,Pitcher & Piano,United Kingdom,gb,England,Nottingham,NG1 1HN,Lace Market,The Meadows,High Pavement,-1.145351,...,NaN,NaN,NaN,openstreetmap,© OpenStreetMap contributors,Open Database Licence,https://www.openstreetmap.org/copyright,"{'bar': 'yes', 'name': 'Pitcher & Piano', 'bra...",http://www.pitcherandpiano.com/where-are-we/no...,"[historic, sights, tourism, building, pub, pla..."
4,Wollaton Dovecote,United Kingdom,gb,England,Nottingham,NG8 2NB,NaN,Wollaton Vale,Dovecote Drive,-1.222958,...,NaN,NaN,NaN,openstreetmap,© OpenStreetMap contributors,Open Database Licence,https://www.openstreetmap.org/copyright,"{'name': 'Wollaton Dovecote', 'owner': 'Nottin...",http://www.ournottinghamshire.org.uk/page_id__...,"[historic, sights, tourism, museum, entertainm..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Eric Irons OBE,United Kingdom,gb,England,Nottingham,NG1 7FG,Lace Market,The Meadows,Carrington Street,-1.147754,...,NaN,48a,NaN,openstreetmap,© OpenStreetMap contributors,Open Database Licence,https://www.openstreetmap.org/copyright,"{'name': 'Eric Irons OBE', 'osm_id': 111428862...",NaN,"[tourism, artwork, attraction]"
190,Nottingham Canal,United Kingdom,gb,England,Nottingham,NG2 3EN,NaN,The Meadows,London Road,-1.139644,...,NaN,NaN,NaN,openstreetmap,© OpenStreetMap contributors,Open Database Licence,https://www.openstreetmap.org/copyright,"{'name': 'Nottingham Canal', 'osm_id': 1115066...",NaN,"[tourism, artwork, attraction]"
191,Mediæval Trent Bridge,United Kingdom,gb,England,Rushcliffe,NG2 5GN,NaN,NaN,London Road,-1.13495,...,West Bridgford,NaN,NaN,openstreetmap,© OpenStreetMap contributors,Open Database Licence,https://www.openstreetmap.org/copyright,"{'ruins': 'yes', 'access': 'no', 'bridge': 'ye...",NaN,"[sights, tourism, heritage, no_access, bridge]"
212,Angel Yard,United Kingdom,gb,England,Nottingham,NG1 1HE,Lace Market,The Meadows,High Pavement,-1.144159,...,NaN,29-31,NaN,openstreetmap,© OpenStreetMap contributors,Open Database Licence,https://www.openstreetmap.org/copyright,"{'osm_id': 2100644353, 'historic': 'entrance',...",NaN,"[tourism, sights]"


In [3]:
# Check NaN rate for each column >0
nan_cols = df.isna().sum()[df.isna().sum()>50]
nan_cols

district       110
old_name       182
village        178
county         160
town           159
housenumber    169
commercial     187
website        177
dtype: int64

In [4]:
postcode_path = '../data/postcode_data/Data/'
# create function for getting postcode data
def get_postcode_data(input_postcode: str, postcode_path) -> pd.DataFrame:
    """Get postcode data from postcode csv files"""
    # match first two letters of postcode to file
    postcode_file = postcode_path + input_postcode[:2] + '.csv'
    # read in file
    postcode_df = pd.read_csv(postcode_file, names=['postcode', 'pqi', 'easting', 'northing'], header=None, usecols=[0, 1, 2, 3])
    #filter to input postcode
    postcode_df = postcode_df[postcode_df['postcode'] == input_postcode]
    # convert easting and northing to longitude and latitude
    postcode_df['longitude'], postcode_df['latitude'] = convert_easting_northing_to_long_lat(postcode_df['easting'].iloc[0], postcode_df['northing'].iloc[0])
    return postcode_df

# test function
get_postcode_data('NG1 1AA', postcode_path)


,postcode,pqi,easting,northing,longitude,latitude
0,NG1 1AA,10,457804,340087,52.955073,-1.141027
